In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from textblob import TextBlob, Word, Blobber
import newspaper
from newspaper import Article
import operator
import rake as rake
from datetime import datetime
rake_object = rake.Rake("SmartStoplist.txt", 1, 2, 1)
import praw

In [7]:
# This is the function in run_praw.py, so I'm using it as a reference for all my algs
def display_praw(name, num):
    reddit = praw.Reddit(client_id='Pj5o8QpNXXJY9A',
                         client_secret='pQKMRBmhp0In48NoNvvktfRo2eA',
                         password='prawisgreat',
                         user_agent='Reddit Unlocked CS196 Project @ UIUC',
                         username='RedditUnlocked196')

    subreddit = reddit.subreddit(name)

    threads_df = pd.DataFrame({
        'Title': (),
        'URL': (),
        'Upvote Ratio (%)': (),
        'Net Score': (),
        '# of Upvotes': (),
        '# of Downvotes': (),
        'Post Date': (),
        'Self Post?': (),
        'Video Post?': (),
        'Domain': ()
    })

    threads_df = threads_df[['Title', 'URL', 'Upvote Ratio (%)', 'Net Score', '# of Upvotes', '# of Downvotes',
                             'Post Date', 'Self Post?', 'Video Post?', 'Domain']]

    for thread in subreddit.top('year', limit=num): # TODO: change limit number when actually deploying program. 15 is the testing number.
        actualUps = int((thread.upvote_ratio * thread.score) / (thread.upvote_ratio * 2 - 1))
        actualDowns = actualUps - thread.score
        gather = pd.Series([thread.title, thread.url, thread.upvote_ratio * 100, thread.score,
                            actualUps, actualDowns, thread.created_utc,
                            thread.is_self, thread.is_video, thread.domain],
                           index=['Title', 'URL', 'Upvote Ratio (%)', 'Net Score', '# of Upvotes', '# of Downvotes',
                                  'Post Date', 'Self Post?', 'Video Post?', 'Domain'])
        threads_df = threads_df.append(gather, ignore_index=True)

    threads_dict = threads_df.to_dict(orient='records')

    for entry in threads_dict:
        if isinstance(str(entry['Post Date']), str):
            time = datetime.fromtimestamp(entry['Post Date'])
            formatTime = time.strftime('%b %d, %Y')
        else:
            formatTime = None

        entry['Post Date'] = formatTime

    return threads_dict


def get_keyword_dict(num):
    # Transforms dict returned by display_praw into DataFrame for working with
    top10news_df = pd.DataFrame.from_dict(display_praw('news', num))

    words = {}

    ## NEWSPAPER STUFF HERE ##

    # Get keywords out of all articles
    for i in range(len(top10news_df)):
        #top10news_df.iloc[i]['url']
        myArticle = Article(top10news_df.iloc[i]['URL'])
        myArticle.download()
        myArticle.parse()
        myArticle.nlp()

        # Run sentiment analysis on each article, fetch subjectivity and polarity
        text = myArticle.text
        blob = TextBlob(text)
        polarity = blob.sentiment.polarity
        subjectivity = blob.sentiment.subjectivity

        # Get associated Reddit post info for each keyword, store in dictionary
        for keyword in myArticle.keywords:

            # Don't waste time with numeric keywords, skip them if they contain numbers
            if any(char.isdigit() for char in keyword):
                continue        

            if keyword not in words:
                words[keyword] = [keyword, 1, 
                                  top10news_df.iloc[i]['# of Upvotes'],
                                  top10news_df.iloc[i]["# of Downvotes"], 
                                  top10news_df.iloc[i]["Net Score"],
                                  subjectivity, polarity, 
                                  {(top10news_df.iloc[i]["Domain"]):1}]
            else:
                words[keyword][1] += 1
                words[keyword][2] += top10news_df.iloc[i]['# of Upvotes']
                words[currentWord][3] += int(top10news_df.iloc[i]['# of Downvotes'])
                words[currentWord][4] += int(top10news_df.iloc[i]['Net Score'])
                words[currentWord][5] = np.mean([subjectivity, words[currentWord][5]])
                words[currentWord][6] = np.mean([polarity, words[currentWord][6]])
                if top10news_df.iloc[i]["Domain"] in words[currentWord][7]:
                    words[currentWord][7][(top10news_df.iloc[i]["Domain"])] += 1
                else:
                    words[currentWord][7][top10news_df.iloc[i]["Domain"]] = 1

        ## RAKE STUFF HERE ##

        # Pull keywords from title strings
        for wordPair in rake_object.run(top10news_df.iloc[i]['Title']):
            currentWord = wordPair[0]

            # Don't waste time with numeric keywords, skip them if they contain numbers
            if any(char.isdigit() for char in currentWord):
                continue

            # Grab associated Reddit post data for each keyword, store in dictionary
            if currentWord not in words:
                words[currentWord] = [currentWord, 1, 
                                  top10news_df.iloc[i]['# of Upvotes'],
                                  top10news_df.iloc[i]["# of Downvotes"], 
                                  top10news_df.iloc[i]["Net Score"],
                                  subjectivity, polarity, 
                                  {(top10news_df.iloc[i]["Domain"]):1}]
            else:
                words[currentWord][1] += 1
                words[currentWord][2] += int(top10news_df.iloc[i]['# of Upvotes'])
                words[currentWord][3] += int(top10news_df.iloc[i]['# of Downvotes'])
                words[currentWord][4] += int(top10news_df.iloc[i]['Net Score'])
                if top10news_df.iloc[i]["Domain"] in words[currentWord][7]:
                    words[currentWord][7][(top10news_df.iloc[i]["Domain"])] += 1
                else:
                    words[currentWord][7][top10news_df.iloc[i]["Domain"]] = 1


    ### FOR GARY'S USE ###
    # Output dictionary is named 'words' #
    # Format is as such: #
    # key = keyword #
    # value = [Occurences, Upvotes, Downvotes, Score, Subjectivity, Polarity, Domain Dictionary] #
    
    return words

In [9]:
import matplotlib as plt
import csv

# Runtime comparison
original_start = datetime.now()

timings = []
for i in range(5, 105, 5):
    start_time = datetime.now()
    get_keyword_dict(i)
    time_taken = datetime.now() - start_time
    timings.append((i, time_taken))
    with open("times.csv", 'a') as f:
        w = csv.writer(f)
        w.writerow([i, time_taken])

final = datetime.now() - original_start
print("Total time taken: %d" % final)

for pair in timings:
    plt.scatter(pair[0], pair[1])
    plt.plot(pair[0], pair[1])
plt.xlabel("Number of Posts Parsed")
plt.ylabel("Processing Time")
plt.show()

Article `download()` failed with 410 Client Error: Gone for url: https://www.nytimes.com/aponline/2017/10/13/us/ap-us-puppy-mills-california.html on URL https://www.nytimes.com/aponline/2017/10/13/us/ap-us-puppy-mills-california.html


ArticleException: 